In [1]:
from pycalphad import Database, Model, variables as v

dbf = Database('alfe_sei.TDB')

mod = Model(dbf, ['AL', 'FE', 'VA'], 'B2_BCC')

In [8]:
from collections import OrderedDict
from pycalphad.core.constraints import build_constraints
res = build_constraints(mod, [v.P, v.T] + mod.site_fractions, OrderedDict([(v.P, 101325), (v.T, 300), (v.N, 1), (v.X('FE'), 0.2)]))

In [9]:
import numpy as np
res.multiphase_jac.kernel(np.array([[101325, 300, 0.5, 0.5, 0.5, 0.5, 1, 1]]), np.array([]))

array([[ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  0.  ,  1.  ],
       [ 0.  ,  0.  , -0.25,  0.25, -0.25,  0.25,  0.  ,  0.5 ]])

In [6]:
from pycalphad import equilibrium

res = equilibrium(dbf, ['AL', 'FE', 'VA'], 'B2_BCC', {v.P: 101325, v.T: 300, v.N: 1, v.X('FE'): 0.2}, verbose=True)

Components: AL FE VA
Phases: B2_BCC [done]
('Redundant phase:', CompositionSet(B2_BCC, [ 0.83967936  0.16032064], NP=0.14999999985029921, GM=-16698.914080687377))
Removing CompositionSet(B2_BCC, [ 0.83967936  0.16032064], NP=nan, GM=-16698.914080687377)
Chemical Potentials [-9342.18619426 -9342.18619426]
[ 0.  0.  0.  0.  0.  0.]
[  8.31663327e-01   1.68336673e-01   1.00000000e+00   1.00000000e-12
   1.00000000e+00   1.00000000e+00]
Status: -10 b'Problem has too few degrees of freedom.'
Adding CompositionSet(B2_BCC, [ 0.5  0.5], NP=0.5, GM=-32185.857895383135) Driving force: 28152.401654502508
Trying to improve poor solution
Calculation Failed:  OrderedDict([('N', array(1.0)), ('P', array(101325.0)), ('T', array(300.0)), ('X_FE', array(0.2))]) b'Maximum number of iterations exceeded (can be specified by an option).'
Chemical Potentials [ 792060.83934283  792060.80336407]
[  2.62824106e+11   7.20055294e-03   2.62824106e+11   7.20055294e-03
   7.20055301e-03   3.04734023e+11   7.20055294

In [5]:
res

<xarray.Dataset>
Dimensions:       (N: 1, P: 1, T: 1, X_FE: 1, component: 2, internal_dof: 5, vertex: 2)
Coordinates:
  * N             (N) float64 1.0
  * vertex        (vertex) int64 0 1
  * X_FE          (X_FE) float64 0.2
  * component     (component) <U2 'AL' 'FE'
  * P             (P) float64 1.013e+05
  * T             (T) float64 300.0
  * internal_dof  (internal_dof) int64 0 1 2 3 4
Data variables:
    GM            (N, P, T, X_FE) float64 nan
    MU            (N, P, T, X_FE, component) float64 nan nan
    Y             (N, P, T, X_FE, vertex, internal_dof) float64 nan nan nan ...
    NP            (N, P, T, X_FE, vertex) float64 nan nan
    X             (N, P, T, X_FE, vertex, component) float64 nan nan nan nan
    Phase         (N, P, T, X_FE, vertex) <U6 '' ''
Attributes:
    engine: pycalphad 0.7+13.ga458f2c.dirty
    created: 2018-04-17T22:59:23.656211

In [11]:
mod.get_multiphase_constraint_contribution(v.N)

(0.5*B2_BCC0AL + 0.5*B2_BCC1AL)/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE) + (0.5*B2_BCC0FE + 0.5*B2_BCC1FE)/(0.5*B2_BCC0AL + 0.5*B2_BCC0FE + 0.5*B2_BCC1AL + 0.5*B2_BCC1FE)